In [68]:
import numpy as np
import pandas as pd

In [82]:
col_names = ['issue_count', 'pulls_count', 'fork_count', 'stars_count', 'branch_count', 'tags_count',]
data = pd.read_csv("github_repos (1).csv",header=None, names = col_names)
#data.drop(['repo_link', 'langs','repo_name','repo_about','repo_about','repo_webs','license','author'], axis = 1, inplace = True)
data = data.iloc[1: , :]
#del data['repo_link', 'langs','repo_name','repo_about','repo_about','repo_webs','license','author']
data.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,,issue_count,pulls_count,fork_count,stars_count,branch_count,tags_count
340.00,131.00,30.7k,79.4k,8.00,129.00,39777.00,1591.00
165.00,66.00,6.3k,19.7k,1.00,51.00,14896.00,528.00
15.00,4.00,914.00,16.9k,9.00,292.00,1964.00,138.00
73.00,6.00,2.7k,16k,4.00,320.00,35920.00,394.00
311.00,39.00,3.3k,13.8k,13.00,28.00,6267.00,365.00


In [88]:
print(data.dtypes)

issue_count     float64
pulls_count     float64
fork_count      float64
stars_count     float64
branch_count    float64
tags_count      float64
dtype: object


In [87]:
#pd.to_numeric(data)
#print(data.dtypes)
#data["issue_count"] = np.where(data["issue_count"][data['issue_count'].apply(len)] == "k",data["issue_count"][0:data['issue_count'].index('k')])
data = data[list(data.columns[~data.columns.duplicated()])]
data['pulls_count'] = pd.to_numeric(data['pulls_count'], errors='coerce')
data['issue_count'] = pd.to_numeric(data['issue_count'], errors='coerce')
data['fork_count'] = pd.to_numeric(data['fork_count'], errors='coerce')
data['stars_count'] = pd.to_numeric(data['stars_count'], errors='coerce')
data['branch_count'] = pd.to_numeric(data['branch_count'], errors='coerce')
data['tags_count'] = pd.to_numeric(data['tags_count'], errors='coerce')
#data['commits_count '] = pd.to_numeric(data['commits_count '], errors='coerce')
#data['contrib_count '] = pd.to_numeric(data['contrib_count '], errors='coerce')

In [89]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [90]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [91]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [94]:
classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_1 <= 32.0 ? 0.23322234672749553
 left:X_4 <= 26.0 ? 0.03379941163693734
  left:X_4 <= 8.0 ? 0.009735147401969846
    left:X_2 <= 1.0 ? 0.002478627447320747
        left:X_4 <= 5.0 ? 0.0017143266415323433
                left:X_3 <= 2.0 ? 0.0008977598854417179
                                left:0.0
                                right:0.0
                right:X_0 <= 0.0 ? 0.0036823072610081276
                                left:0.0
                                right:0.0
        right:X_4 <= 3.0 ? 0.005772907599559451
                left:X_0 <= 0.0 ? 0.005871085266938275
                                left:0.0
                                right:0.0
                right:X_2 <= 8.0 ? 0.0042500881378281785
                                left:0.0
                                right:0.0
    right:X_0 <= 0.0 ? 0.009948577655206403
        left:X_2 <= 1.0 ? 0.0061291014810984645
                left:X_1 <= 27.0 ? 0.0009169756154609465
                                left:0.0

In [95]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

ValueError: ignored